### 인스타그램 크롤링

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as ES
from selenium.webdriver.support.wait import WebDriverWait

# 0. 로그인
options = Options()
options.add_argument("user-data-dir=C:\\Users\\dxsch\\web_crawling_study\\selenium_options")


options.add_argument("disable-blink-features=AutomationControlled") 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.get("https://www.instagram.com/")

# 1. 검색창 클릭
searchBtn = driver.find_element(By.CSS_SELECTOR, ".x1iyjqo2.xh8yej3>div:nth-child(2)>span>div>a")
searchBtn.click()

# 2. 검색
searchInputSelector=".x1lugfcp.x19g9edo.x1lq5wgf.xgqcy7u.x30kzoy.x9jhf4c.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x5n08af.xl565be.x5yr21d.x1a2a7pz.xyqdw3p.x1pi30zi.xg8j3zb.x1swvt13.x1yc453h.xh8yej3.xhtitgo.xs3hnx8.x1dbmdqj.xoy4bel.x7xwk5j"
searchInput=driver.find_element(By.CSS_SELECTOR,searchInputSelector)
searchInput.send_keys("김수현")

# 3. 첫번째 계정 클릭
searchList = ".x1i10hfl.x1qjc9v5.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x87ps6o.x1lku1pv.x1a2a7pz.x1dm5mii.x16mil14.xiojian.x1yutycm.x1lliihq.x193iq5w.xh8yej3:first-child"
WebDriverWait(driver, 10).until(ES.presence_of_all_elements_located((By.CSS_SELECTOR, searchList)))
searchFirstNode = driver.find_element(By.CSS_SELECTOR, searchList)
searchFirstNode.click()


In [2]:
# 4-1. 스크롤 페이징 불러오기(12번째 이상 블럭 부터 찾아지지 않는 문제 발생)
imgBlockSelector = "._ac7v.xras4av.xgc1b0m.xat24cr.xzboxd6"
WebDriverWait(driver, 10).until(ES.presence_of_all_elements_located((By.CSS_SELECTOR, imgBlockSelector+ ":first-child")))
body = driver.find_element(By.TAG_NAME, "body")

while True:
    # 현재 출력된 이미지 블럭 모두 찾기
    imgBlocks = driver.find_elements(By.CSS_SELECTOR, imgBlockSelector)  # 없으면 []
    imgBlcoksLen = f":nth-child({len(imgBlocks)+1})"  # 다음블럭 선택자
    body.send_keys(Keys.END)

    try:
        WebDriverWait(driver, 10).until(ES.presence_of_all_elements_located((By.CSS_SELECTOR, imgBlockSelector + imgBlcoksLen)))
        print(imgBlcoksLen)
        print("로딩중")
    except:
        print("모든 이미지가 로딩되었습니다.")
        break


In [3]:
# 4-2.
time.sleep(2)

while True:
    
    prevHeight = driver.execute_script("return document.body.scrollHeight")  # 이전
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)

    nowHeight = driver.execute_script("return document.body.scrollHeight")  # 이후
    
    if prevHeight == nowHeight:
        print("모든 이미지가 로딩되었습니다.")
        break

모든 이미지가 로딩되었습니다.


In [8]:
# 5. 이미지 저장
import os # 폴더 운영체제가 만든다.
from urllib.request import urlretrieve  # 리소스의 주소로 이미지를 다운받는다.
from bs4 import BeautifulSoup

soup = BeautifulSoup(driver.page_source, "html.parser")
imgs = soup.select("._aagv>img")
imgDir = "C:\\Users\\dxsch\\web_crawling_study\\imgs\\soohyun_k216"

if not os.path.isdir(imgDir):  # 해당 경로에 폴더가 없으면 
    os.mkdir(imgDir)

i = 0
for img in imgs:  # [img(class, src, id, alter...), img, ....]
    i += 1
    src = img.get("src")  # bs.get() 속성의 값을 가져오는 함수
    urlretrieve(src, imgDir + "\\soohyun_k216" + str(i) + ".jpeg")

